In [ ]:
import synapseclient

import great_expectations as gx
import pandas as pd
import json

context = gx.get_context(project_root_dir='../src/agoradatatools/great_expectations')

from agoradatatools.gx import GreatExpectationsRunner
from expectations.expect_column_values_to_have_list_length import ExpectColumnValuesToHaveListLength
from expectations.expect_column_values_to_have_list_members import ExpectColumnValuesToHaveListMembers
from expectations.expect_column_values_to_have_list_members_of_type import ExpectColumnValuesToHaveListMembersOfType

# Create Expectation Suite for Pathology Data

In [ ]:
syn = synapseclient.Synapse()
syn.login()


In [ ]:
pathology_data_file = syn.get("syn63644533").path

## Create Validator Object on Data File

In [ ]:
df = pd.read_json(pathology_data_file)
nested_columns = ['points']
df = GreatExpectationsRunner.convert_nested_columns_to_json(df, nested_columns)
validator = context.sources.pandas_default.read_dataframe(df)
validator.expectation_suite_name = "pathology"

## Add Expectations to Validator Object For Each Column

In [ ]:
# points
validator.expect_column_values_to_be_of_type("points", "list")
with open("../src/agoradatatools/great_expectations/gx/json_schemas/immunohisto/points.json", "r") as file:
    points_schema = json.load(file)
validator.expect_column_values_to_match_json_schema("points", json_schema=points_schema)

In [ ]:
# model
validator.expect_column_values_to_be_of_type("model", "str")
validator.expect_column_values_to_not_be_null("model")
# allows all alphanumeric characters, underscores, periods, and dashes
validator.expect_column_values_to_match_regex("type", "^[A-Za-z0-9\s\(\)\*_.-]+$")


In [ ]:
# type
validator.expect_column_values_to_be_of_type("type", "str")
validator.expect_column_values_to_not_be_null("type")
# allows all alphanumeric characters, underscores, periods, and dashes
validator.expect_column_values_to_match_regex("type", "^[A-Za-z0-9\s\(\)\*_.-]+$")

In [ ]:
# units
validator.expect_column_values_to_be_of_type("units", "str")
validator.expect_column_values_to_not_be_null("units")
# allows all alphanumeric characters, underscores, periods, and dashes
validator.expect_column_values_to_match_regex("units", "^[A-Za-z0-9\s\(\)\#\/\\%*_.-]+$")

In [ ]:
# age_death
validator.expect_column_values_to_be_of_type("age_death", "int")
validator.expect_column_values_to_not_be_null("age_death")
validator.expect_column_values_to_be_between("age_death", strict_min_value=0, max_value=100)

In [ ]:
# tissue
validator.expect_column_values_to_be_of_type("tissue", "str")
validator.expect_column_values_to_not_be_null("tissue")
# allows all alphanumeric characters, underscores, periods, and dashes
validator.expect_column_values_to_match_regex("tissue", "^[A-Za-z0-9\/\s\*_.-]+$")

In [ ]:
# unique entries ExpectSelectColumnValuesToBeUniqueWithinRecord
validator.expect_select_column_values_to_be_unique_within_record(column_list=["model", "type", "age_death", "tissue", "units"])

In [ ]:
validator.save_expectation_suite(discard_failed_expectations=False)


## Create Checkpoint and View Results

In [ ]:
checkpoint = context.add_or_update_checkpoint(
    name="agora-test-checkpoint",
    validator=validator,
)
checkpoint_result = checkpoint.run()
context.view_validation_result(checkpoint_result)


## Build Data Docs - Click on Expectation Suite to View All Expectations

In [ ]:
context.build_data_docs()
context.open_data_docs()
